## Azure OpenAI Service - Data Exploration Example

To run this sample notebook, you will need to have an instance of Azure OpenAI Service provisioned. You will need to retrieve your API key, service URI, and name of a deployed model you aim to target for execution of this notebook.

Import required packages

In [0]:
import openai
import numpy as np
import os
from pyspark.sql import SparkSession  
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType  
from sklearn.datasets import load_iris  

Initialize Spark session

In [0]:
spark = SparkSession.builder \
    .appName("Delta Table with scikit-learn dataset") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()


Load toy dataset from scikit-learn and define schema

In [0]:
iris = load_iris()  
data = iris['data']  
feature_names = iris['feature_names']
targets = iris['target']
targets = targets.reshape(len(data), 1)

data = np.concatenate((data, targets), axis=1)

schema = StructType([  
    StructField("sepal_length", DoubleType(), True),  
    StructField("sepal_width", DoubleType(), True),  
    StructField("petal_length", DoubleType(), True),  
    StructField("petal_width", DoubleType(), True),  
    StructField("target", DoubleType(), True), 
])


Create a Spark DataFrame and save as a Delta table

In [0]:
data_df = spark.createDataFrame(data.tolist(), schema=schema)   
data_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/tmp/iris-delta-table") 


Set `OPENAI_API_KEY` and `OPENAI_BASE` as environment variables.

In [0]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR-KEY'
os.environ['OPENAI_BASE'] = 'YOUR-AOAI-SERVICE-URI' # should have format https://yourservicename.openai.azure.com/

OpenAI prompt helper function.

<i>Note:</i> Update the `engine` field in the OpenAI chat completion prompt with the name of a deployed model in your service

In [0]:
def prompt_openai(prompt):
    import os
    import openai
    openai.api_type = "azure"
    openai.api_base = os.getenv("OPENAI_BASE")
    openai.api_version = "2023-03-15-preview"
    openai.api_key = os.getenv("OPENAI_API_KEY")
    
    response = openai.ChatCompletion.create(
      engine="YOUR-DEPLOYMENT-NAME",
      messages = [{"role":"system","content":"You are an AI assistant that helps people get information."},{"role":"user","content":prompt}],
      temperature=0.0,
      max_tokens=3000,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None)
    
    return response['choices'][0]['message']['content']

Use OpenAI to help query data from Delta Lake

In [0]:
prompt = """
TABLE SCHEMA: sepal_width, sepal_length, petal_length, petal_width, target. Write me a spark SQL command that will query all data from my table where sepal_length is > 5  from delta table at '/tmp/iris-delta-table'
"""

response = prompt_openai(prompt)
print(response)

Execute returned query

In [0]:
%sql
SELECT sepal_width, sepal_length, petal_length, petal_width, target
FROM delta.`/tmp/iris-delta-table`
WHERE sepal_length > 5

Convert Spark DataFrame to Pandas

In [0]:
pandas_df = _sqldf.toPandas()

Use OpenAI to summarize data.

In [0]:
prompt = f"""
From the table of data below, describe general trends for inclusion in an executive report.

DATA:{pandas_df.to_json(orient='records')}
"""

response = prompt_openai(prompt)
print(response)

Use OpenAI to create an analysis function

In [0]:
prompt = f"""
Create and return a python function to calculate the average value of 'sepal_width', 'sepal_length', and 'petal_length' associated with each unique 'target' value. Assume the data exists in a pandas dataframe named 'pandas_df'.
"""

response = prompt_openai(prompt)
print(response)

Execute analysis function

In [0]:
import pandas as pd

def calculate_average(df):
    # Group the dataframe by the 'target' column and calculate the mean of the specified columns
    average_values = df.groupby('target')[['sepal_width', 'sepal_length', 'petal_length']].mean()
    
    return average_values

# Assuming you have a pandas DataFrame named 'pandas_df'
# Call the function with the 'pandas_df' DataFrame
result = calculate_average(pandas_df)

# Print the result
print(result)